In [377]:
import pickle
import json
import numpy as np
from operator import itemgetter
from nltk.tokenize import word_tokenize

In [336]:
tfhub_data_path = '/home/olga/Documents/drones_paragraph_ranking.pkl'
rnet_data_path = '/home/olga/Documents/r_net_ranker_scores.pckl'
tfidf_data_path = '/media/olga/Data/projects/ODQA/data/PMEF/ranker1_drones_all_paragraphs.json'

In [337]:
def load_pickle(path):
    with open(path, 'rb') as fin:
        return pickle.load(fin)

In [338]:
def load_json(path):
    with open(path) as fin:
        return json.load(fin)

In [339]:
tfhub_data = load_pickle(tfhub_data_path)
rnet_data = load_pickle(rnet_data_path)
tfidf_data = load_json(tfidf_data_path)

In [340]:
len(tfhub_data)

149

In [341]:
len(rnet_data)

137

In [342]:
len(tfidf_data)

149

In [343]:
tfhub_data[0]

('what country manufactured the helicopter?',
 'Russia',
 [('In early 2017, the news arose that the management of unmanned aerial vehicles was officially recognized in Russia as a profession, for the development of which it would be necessary to undergo training at special courses.',
   0.56699222),
  ('\ufeffThe Helicopters of Russia (part of the Rostekh state corporation) has developed a VRT300 unmanned vehicle with a maximum takeoff weight of 300 kg and a target payload of 70 kg.',
   0.56459445),
  ('According to the definition approved by the ICAO Assembly, an unmanned aerial vehicle (drone) is an aircraft without a pilot who is flying without an aircraft commander on board and is either completely remotely controlled from another location from the ground, from another aircraft, from space, or programmed and fully autonomous.',
   0.5570243),
  ("The presented version of VRT300 is intended for collection and is one of the consequences of Sberbank's partnership and Russian Helicopt

In [344]:
rnet_data[0]

('what country manufactured the helicopter?',
 'Russia',
 [(0.9978230595588684,
   'Helicopters of Russia JSC (part of Rostekh State Corporation) is one of the world leaders in the helicopter industry, the only developer and manufacturer of helicopters in Russia. The holding was established in 2007. The head office is located in Moscow. The holding includes five helicopter factories, two design bureaus, as well as enterprises for the production and maintenance of components, aircraft repair plants and a service company that provides after-sales support in Russia and abroad.'),
  (0.8298145532608032,
   'The St. Petersburg International Economic Forum is scheduled to sign a Memorandum of Cooperation between PJSC Sberbank and JSC Helicopters of Russia, within the framework of this memorandum, it is planned to establish partnerships aimed at creating a federal logistics operator using unmanned aerial systems of a helicopter type.'),
  (0.792402982711792,
   "The Bank's foreign network con

In [345]:
tfidf_data[0]

['what country manufactured the helicopter?',
 'Russia',
 [['Helicopters of Russia JSC (part of Rostekh State Corporation) is one of the world leaders in the helicopter industry, the only developer and manufacturer of helicopters in Russia. The holding was established in 2007. The head office is located in Moscow. The holding includes five helicopter factories, two design bureaus, as well as enterprises for the production and maintenance of components, aircraft repair plants and a service company that provides after-sales support in Russia and abroad.',
   12.161507133848627],
  ['Maintenance and maintenance of UAVs is cheaper than similar costs for manned aircraft. After all, planes and helicopters need to maintain safety systems and protect pilots. Specialists managing and servicing aircraft and helicopters must undergo training, retraining, and medical commissions. The time and financial costs for drones are incommensurably lower.',
   3.9411082975728586],
  ["The organizations of R

In [346]:
# make paragraph: id map

all_pars = []
for instance in tfhub_data:
    pars = instance[2]
    all_pars += [p[0] for p in pars]

all_pars = list(set(all_pars))

In [347]:
len(all_pars)

58

In [348]:
par2id = {all_pars[i]: i for i in range(len(all_pars))}

In [349]:
id2par = {v: k for k, v in par2id.items()}

In [350]:
all_pars_rnet = []
for instance in rnet_data:
    pars = instance[2]
    all_pars_rnet += [p[0] for p in pars]

all_pars_rnet = list(set(all_pars))

for p in all_pars_rnet:
    if p not in par2id.keys():
        print(p)
else:
    print('All pars exist.')

All pars exist.


In [351]:
all_pars_tfidf = []
for instance in tfidf_data:
    pars = instance[2]
    all_pars_tfidf += [p[0] for p in pars]

all_pars_tfidf = list(set(all_pars))

for p in all_pars_tfidf:
    if p not in par2id.keys():
        print(p)
else:
    print('All pars exist.')

All pars exist.


In [352]:
all_questions = [item[0] for item in rnet_data]

In [353]:
all_questions

['what country manufactured the helicopter?',
 'in which year was the helicopter created?',
 'weight of a drones?',
 'how long does the helicopter work?',
 'What are the dimensions of this model of unmanned helicopter?',
 'where you can buy?',
 'where you can buy a helicopter?',
 'where is the helicopter?',
 'where is it?',
 'What is the carrying capacity of a helicopter?',
 'What is the weight of the helicopter?',
 'What is the mass of the helicopter?',
 'What time can be in the air?',
 'what amount of payload is it capable of carrying?',
 'what is the weight of the helicopter?',
 'how long can the helicopter be in the air?',
 'how long is the helicopter capable of being in the air?',
 'how long is the duration of a helicopter flight?',
 'how much does the helicopter weigh?',
 'how long can a it stay in the air without moving around ?',
 'how many types of unmanned helicopters are there?',
 'how many types does it have?',
 'how many wings it has?',
 'what is the average weight of an u

In [383]:
# modify anaphopric questions

anaphora_tokens = ['it', 'this', 'these', 'they']
new_questions = []
prev_tokens = []
for i in range(len(all_questions)):
    q = all_questions[i]
    cur_tokens = word_tokenize(q)
    if (any(t in cur_tokens for t in anaphora_tokens)):
        if prev_tokens:
            new_question = ' '.join(set(cur_tokens + prev_tokens))
            print(new_question)
            new_questions.append(new_question)
        else:
            new_questions.append(all_questions[i])
    else:
        new_questions.append(all_questions[i])
        prev_tokens = cur_tokens
    
    

dimensions the of helicopter does What model this how work long are unmanned ?
the helicopter is where it ?
air of what payload be the What amount time capable is can it carrying in ?
air the does helicopter a around much moving how without can long in weigh it stay ?
have does of ? helicopters how there types it are unmanned many
of ? helicopters how there has types wings it are unmanned many
the what of helicopter ? weight average is an benefit unmanned it
the what of helicopter functions this function is an unmanned ?
the what of helicopter is an purpose it unmanned ?
the what of weight is it ?
the what helicopter with for time this is full working charge ?
the what of 's with time charge is it full aim working ?
be the helicopter somewhere bought can begin when it did production unmanned ?
the what of helicopter control name this how is to ?
of helicopter a What drone is developed kind it company which ?
the of helicopter What country for time used is speed it which first ?
the of 

In [385]:
len(new_questions)

137

In [354]:
all_answers = [item[1] for item in rnet_data]

In [521]:

mean_rankers_data = []

for q, a in zip(all_questions, all_answers):
#     print(q)
    tfhub_par_scores = [item[2] for item in tfhub_data if item[0]==q][0]
    tfhub_par_id_scores = {par2id[pair[0]]: pair[1] for pair in tfhub_par_scores}
    rnet_par_scores = [item[2] for item in rnet_data if item[0]==q][0]
    rnet_par_id_scores = {par2id[pair[1]]: pair[0] for pair in rnet_par_scores}
    tfidf_par_scores = [item[2] for item in tfidf_data if item[0]==q][0]
    tfidf_par_id_scores = {par2id[pair[0]]: pair[1] for pair in tfidf_par_scores}
    
    tfidf_norm = np.linalg.norm(list(tfidf_par_id_scores.values()))
    tfidf_par_id_scores = {k: v / norm for k, v in tfidf_par_id_scores.items()}
    
#     print(tfhub_par_id_scores)
#     print(rnet_par_id_scores)
#     print(tfidf_par_id_scores)
    
    idx2mean = []
    for idx in par2id.values():
        tfhub_score = tfhub_par_id_scores[idx]
        rnet_score = rnet_par_id_scores[idx]
        tfidf_score = tfidf_par_id_scores[idx]
        mean_score = np.mean([tfhub_score, rnet_score, tfidf_score])                        
#         print(idx)
#         print(np.mean([tfhub_score, rnet_score, tfidf_score]))
        idx2mean.append((idx, mean_score))
    idx2mean.sort(key=itemgetter(1))
    idx2mean = idx2mean[::-1]
    
    mean_rankers_data.append((q, a, idx2mean))
    

In [522]:
id2par[2]

'Helicopters of Russia JSC (part of Rostekh State Corporation) is one of the world leaders in the helicopter industry, the only developer and manufacturer of helicopters in Russia. The holding was established in 2007. The head office is located in Moscow. The holding includes five helicopter factories, two design bureaus, as well as enterprises for the production and maintenance of components, aircraft repair plants and a service company that provides after-sales support in Russia and abroad.'

In [523]:
mean_rankers_data[1]

('in which year was the helicopter created?',
 '2018',
 [(2, 0.49433278681240861),
  (17, 0.31114465049027873),
  (56, 0.30046605621210737),
  (4, 0.27306385392379762),
  (15, 0.26055708947188078),
  (30, 0.25630342772758491),
  (27, 0.25094278954243399),
  (50, 0.23232590034676179),
  (6, 0.22949229553413972),
  (41, 0.22194204154020966),
  (53, 0.20412180378468833),
  (29, 0.19951399043274506),
  (44, 0.17758140518697105),
  (43, 0.17708038841120402),
  (49, 0.17585070803833588),
  (10, 0.1645140206260681),
  (33, 0.16348982408078511),
  (18, 0.16337793941371223),
  (39, 0.16257499212455748),
  (38, 0.16201468609547354),
  (47, 0.15862867431961761),
  (13, 0.15658387099138896),
  (31, 0.1538787678582991),
  (52, 0.15290924413755977),
  (35, 0.14955179447364805),
  (26, 0.14651575241915385),
  (42, 0.14480520588930187),
  (9, 0.14230040266545613),
  (37, 0.1405223464412689),
  (8, 0.14044505977050781),
  (3, 0.13966574822298686),
  (16, 0.13843724682362873),
  (51, 0.13669546002578734

In [524]:
def instance_score(answer, texts):
    for doc_text in texts:
        if doc_text.find(answer) != -1:
            return 1
    return 0

In [525]:
recall_mapping = {}
for n in range(1, 59):
    correct_answers = 0
    for instance in mean_rankers_data:
        top_n_ids = [pair[0] for pair in instance[2]][:n]
        top_n_texts = [id2par[j] for j in top_n_ids]
        top_n_texts = [text.lower() for text in top_n_texts]
        formatted_answer = instance[1].strip().lower()
        correct_answers += instance_score(formatted_answer, top_n_texts)
    recall = correct_answers / len(rnet_data)
    recall_mapping[n] = recall

In [526]:
recall_mapping

{1: 0.34306569343065696,
 2: 0.44525547445255476,
 3: 0.48905109489051096,
 4: 0.5328467153284672,
 5: 0.5474452554744526,
 6: 0.5766423357664233,
 7: 0.6131386861313869,
 8: 0.6423357664233577,
 9: 0.6715328467153284,
 10: 0.7007299270072993,
 11: 0.7153284671532847,
 12: 0.7226277372262774,
 13: 0.7445255474452555,
 14: 0.7591240875912408,
 15: 0.7883211678832117,
 16: 0.7956204379562044,
 17: 0.8102189781021898,
 18: 0.8394160583941606,
 19: 0.8394160583941606,
 20: 0.8394160583941606,
 21: 0.8467153284671532,
 22: 0.8613138686131386,
 23: 0.8832116788321168,
 24: 0.8905109489051095,
 25: 0.8978102189781022,
 26: 0.8978102189781022,
 27: 0.9051094890510949,
 28: 0.9124087591240876,
 29: 0.9124087591240876,
 30: 0.9124087591240876,
 31: 0.9124087591240876,
 32: 0.9124087591240876,
 33: 0.9197080291970803,
 34: 0.927007299270073,
 35: 0.927007299270073,
 36: 0.9343065693430657,
 37: 0.9343065693430657,
 38: 0.9343065693430657,
 39: 0.9416058394160584,
 40: 0.9416058394160584,
 41: 0.9